In [1]:
import tensorflow as tf
from tensorflow import keras

### Simple Food Classifcation model

In [ ]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from os.path import exists
from collections import defaultdict

def load_data(file_path, base_dir, max_per_class=None):
    image_paths = defaultdict(list)
    labels = []
    
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.split('/')
            class_name = parts[0].strip()
            img_path = f"{base_dir}/{class_name}/{parts[1].strip()}"
            image_paths[class_name].append(img_path)
    
    # Restrict the number of images per class
    final_image_paths = []
    final_labels = []
    for class_name, paths in image_paths.items():
        if max_per_class:
            paths = paths[:max_per_class]  # Limit to max_per_class images
        final_image_paths.extend(paths)
        final_labels.extend([class_name] * len(paths))
    
    return final_image_paths, final_labels

def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for img_path in image_paths:
        img_path = img_path + '.jpg'
        if not exists(img_path):
            print(f"Image path does not exist: {img_path}")
            continue
        img = Image.open(img_path)
        img = img.convert('RGB')  # Ensure all images are in RGB mode
        img = img.resize(target_size)
        img_array = np.array(img)
        
        # Check if the image shape is correct
        if img_array.shape != (target_size[0], target_size[1], 3):
            print(f"Unexpected image shape: {img_array.shape} for image {img_path}")
            continue
        
        images.append(img_array)
    return np.array(images)

# Load training and testing data with fewer files per class
train_image_paths, train_labels = load_data(
    '/Users/aariziqbal/Desktop/Homework/ECS_174/project/ECS_174_FoodClassification/food-101/meta/train.txt',
    '/Users/aariziqbal/Desktop/Homework/ECS_174/project/ECS_174_FoodClassification/food-101/images',
    max_per_class=80  # Take at most 50 images per class
)
test_image_paths, test_labels = load_data(
    '/Users/aariziqbal/Desktop/Homework/ECS_174/project/ECS_174_FoodClassification/food-101/meta/test.txt',
    '/Users/aariziqbal/Desktop/Homework/ECS_174/project/ECS_174_FoodClassification/food-101/images',
    max_per_class=20  # Take at most 20 images per class
)

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Preprocess images
print('train_image_paths:', train_image_paths)
train_images = load_and_preprocess_images(train_image_paths)
test_images = load_and_preprocess_images(test_image_paths)

# Normalize images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert to NumPy arrays for model input
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels_encoded = np.array(train_labels_encoded)
test_labels_encoded = np.array(test_labels_encoded)

print(f"Classes: {label_encoder.classes_}")


In [4]:
## using the data in a Tensorflow tensor
import tensorflow as tf
# Define a data augmentation function
def data_augmentation(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)  # Random brightness adjustment
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Random contrast adjustment
    image = tf.image.random_crop(image, size=[224, 224, 3])  # Random crop (adjust size as needed)
    return image, label

# Apply data augmentation to the training dataset
train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_images, train_labels_encoded))
    .map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize images to 224x224
    .map(data_augmentation, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(buffer_size=1000)
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Preprocess the test dataset (no augmentation for testing)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((test_images, test_labels_encoded))
    .map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize images to 224x224
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


: 

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the model
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224, 224, 3)),
        
        # First conv block
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        # Second conv block
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        # Third conv block
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2),
        
        # Dense layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(101, activation='softmax')
    ])

# Compile the model



Epoch 1/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 252s 598ms/step - accuracy: 0.0371 - loss: 11.1513 - val_accuracy: 0.0087 - val_loss: 4.7259
Epoch 2/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 244s 587ms/step - accuracy: 0.0159 - loss: 4.5815 - val_accuracy: 0.0062 - val_loss: 4.8330
Epoch 3/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 251s 605ms/step - accuracy: 0.0167 - loss: 4.6293 - val_accuracy: 0.0099 - val_loss: 4.6157
Epoch 4/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 1746s 4s/step - accuracy: 0.0160 - loss: 4.5929 - val_accuracy: 0.0099 - val_loss: 4.6155
Epoch 5/10
226/411 ━━━━━━━━━━━━━━━━━━━━ 6:18 2s/step - accuracy: 0.0230 - loss: 4.5839

KeyboardInterrupt: 

In [1]:
from tensorflow.keras import optimizers, models
def create_model():
    # Load pre-trained EfficientNetB0
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(224, 224, 3)),
            
            # First conv block
            tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(2),
            
            # Second conv block
            tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(2),
            
            # Third conv block
            tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(2),
            
            # Dense layers
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(101, activation='softmax')
        ])
    
    return model

model = create_model()

model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train the model
history_improved = model.fit(train_dataset,
                                      validation_data=test_dataset,
                                      epochs=30)


NameError: name 'tf' is not defined

In [7]:
print(history_improved.history)

{'accuracy': [0.06031987816095352, 0.06176694482564926, 0.058111194521188736, 0.0664127916097641, 0.06466107815504074, 0.07204874604940414, 0.08225437998771667, 0.09619192779064178, 0.1030464619398117, 0.10281797498464584], 'loss': [5.001447677612305, 4.865840435028076, 4.777949810028076, 4.7050652503967285, 4.644796371459961, 4.537500858306885, 4.449641227722168, 4.337552547454834, 4.236262321472168, 4.206655025482178], 'val_accuracy': [0.009900989942252636, 0.009900989942252636, 0.014108911156654358, 0.013861386105418205, 0.013366336934268475, 0.010396040044724941, 0.010396040044724941, 0.009405940771102905, 0.014603960327804089, 0.035396039485931396], 'val_loss': [4.8968682289123535, 590.5326538085938, 4.738389015197754, 4.883778095245361, 4.916850566864014, 6.653088092803955, 5.8490142822265625, 5.189481258392334, 5.158570766448975, 4.755159854888916]}
